# 入試合格判定用コード

PDF で合格者の受験番号が特定の Web ページに掲示される形式の入試の合否が判定できます。  
「以下を必要に応じて変更してください」の部分を変更し、セルを上から実行することで、自分の受験番号が合格者の一覧にあるかを見ることができます。  
なお、合否が正しく表示されなかった場合に生じる不利益については、責任を負いかねますのでご了承ください。

In [13]:
### 以下を必要に応じて変更してください ###

# ターゲットURL（合格者の受験番号のPDFリンクが貼られるページのURL）
# 例：東大薬学系研究科は https://www.f.u-tokyo.ac.jp/exam/index.html?key=1378943625
url = ""

# リンクテキスト（ページ内で、PDFのリンク部分が一意に定まるようにしてください）
# 例：東大薬学系研究科修士課程は 修士課程合格者、2025.4
linktext1 = ""
linktext2 = ""

# PDFのURLのうち、aタグに含まれていない部分
page = ""

# 受験番号
search_string = ""

## ①モジュールのインポートとか

In [5]:
!pip freeze | grep request
!pip freeze | grep beautiful
!pip install PyPDF2
import requests
from bs4 import BeautifulSoup
import urllib.request
from PyPDF2 import PdfReader
import ssl
import io

requests==2.32.3
requests-oauthlib==1.3.1
requests-toolbelt==1.0.0
beautifulsoup4==4.12.3


## ②結果の PDF の URL 取得

ここで「該当するリンクが見つかりませんでした」と表示された場合、ファイルの命名規則が変わったなどの理由で URL が開けなくなっています。その場合、正しい合否の判定ができません 🥺

In [8]:
# ウェブページを取得
response = requests.get(url, verify=False)

# エンコーディングを自動検出
response.encoding = response.apparent_encoding

# BeautifulSoupでHTMLをパース
html = BeautifulSoup(response.text, "html.parser")

# 'a'タグを検索し、リンクテキストの一部が一致するかチェック
link = html.find("a", string=lambda text: text and linktext1 in text and linktext2 in text)

# 結果を出力
if link:
    href = link.get('href')
    print(href)
else:
    print("該当するリンクが見つかりませんでした")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.f.u-tokyo.ac.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/wysiwyg/kcfinder/upload/files/%EF%BC%92%EF%BC%90%EF%BC%92%EF%BC%95%E5%B9%B4%EF%BC%94%E6%9C%88%E5%85%A5%E5%AD%A6%E3%80%80%E4%BF%AE%E5%A3%AB%E8%AA%B2%E7%A8%8B%E3%80%80%E5%90%88%E6%A0%BC%E8%80%85.pdf


# ③ PDF に特定の受験番号があるか確認

### 関数の定義

In [9]:
# PDFをダウンロードして読み込む関数
def download_pdf(url):
    context = ssl._create_unverified_context()  # SSL証明書の検証を無視（同上）
    response = urllib.request.urlopen(url, context=context)
    return response.read()

# PDFのページごとに文字列を検索する関数
def search_string_in_pdf(pdf_data, search_string):
    pdf_file = io.BytesIO(pdf_data)  # バイトデータをファイルのように扱う
    pdf_reader = PdfReader(pdf_file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        if search_string in text:
            print(f"文字列が{page_num + 1}ページ目で見つかりました")
            return "合格です！おめでとう〜！"
    print("String not found in the PDF")
    return "不合格です🥺"

### 合否の判定

In [12]:
# URLからPDFを取得
pdf_data = download_pdf(page + href)

# 特定の文字列を検索
search_string_in_pdf(pdf_data, search_string)

文字列が3ページ目で見つかりました


'合格です！おめでとう〜！'